In [1]:
import nibabel as nib
import numpy as np
import pandas as pd
from neuromaps import datasets, images, parcellate


def relabel_gifti(gifti):
    """Updates GIFTI images so label IDs are consecutive integers.

    Parameters
    ----------
    gifti : (2,)
        Surface label files in GIFTI format (lh.label.gii, rh.label.gii).

    Returns
    -------
    (2,) tuple-of-nib.gifti.GiftiImage
        Relabeled `gifti` images.
    """

    gifti_l, gifti_r = gifti[0].agg_data(), gifti[1].agg_data()

    unique_labels = np.unique(np.concatenate([gifti_l, gifti_r]))
    label_map = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}
    label_map = np.vectorize(label_map.get)

    gifti_relabelled_l, gifti_relabelled_r = label_map(gifti_l), label_map(gifti_r)

    # make new gifti image with updated labels
    gifti_relabelled = tuple()
    for hemi in (gifti_relabelled_l, gifti_relabelled_r):
        darray = nib.gifti.GiftiDataArray(
            hemi, intent="NIFTI_INTENT_LABEL", datatype="NIFTI_TYPE_INT32"
        )
        labeltable = nib.gifti.GiftiLabelTable()
        labeltable.labels = np.unique(hemi)
        img = nib.GiftiImage(darrays=[darray], labeltable=labeltable)
        gifti_relabelled += (img,)

    return gifti_relabelled


def parcellate_all_annotations(gifti_relabeled):
    """Parcellate all available neuromaps annotations and return them as a pandas DataFrame.

    Parameters
    ----------
    gifti_relabeled : (2,) tuple-of-nib.gifti.GiftiImage
        GIFTI file that has been relabeled.

    Returns
    -------
    pandas.DataFrame of shape (n_parcels, n_annotations)
        A DataFrame containing all available annotations.
    """

    parcellation = parcellate.Parcellater(
        gifti_relabeled, space="fsLR", resampling_target="parcellation"
    )

    # list of annotations to skip
    skip = [("beliveau2017", "MNI152"), ("hill2010", "fsLR"), ("norgaard2021", "MNI152")]

    df = dict()
    for annotation in datasets.available_annotations():
        kwargs = {key: value for key, value in zip(["source", "desc", "space", "res"], annotation)}

        # skip annotations in list `skip`
        if (annotation[0], annotation[2]) in skip:
            print(f"x {kwargs}")
            continue

        map = datasets.fetch_annotation(**kwargs)
        map_vec = parcellation.fit_transform(map, space=kwargs["space"])
        df[f"{kwargs['source']}-{kwargs['desc']}"] = map_vec

    return pd.DataFrame(df)

/usr/local/Caskroom/miniconda/base/envs/fmri-timescales/lib/python3.11/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
# cifti surface+volumetric parcellation file
dlabel = "./data/nregions-300_cifti.dlabel.nii"

# cifti -> surface removes volumetric data: 300 -> 106 parcels
gifti = images.dlabel_to_gifti(dlabel)
gifti_relabeled = relabel_gifti(gifti)

# get dataframe of annotations
try:
    df = pd.read_csv("./data/nregions-106_annotations.csv")
except FileNotFoundError:
    df = parcellate_all_annotations(gifti_relabeled)

df.head()

,abagen-genepc1,aghourian2017-feobv,alarkurtti2015-raclopride,bedard2019-feobv,beliveau2017-az10419369,beliveau2017-cimbi36,beliveau2017-cumi101,beliveau2017-dasb,beliveau2017-sb207145,ding2010-mrb,...,savli2012-dasb,savli2012-p943,savli2012-way100635,smart2019-abp688,smith2017-flb457,sydnor2021-SAaxis,tuominen-feobv,turtonen2020-carfentanil,xu2020-FChomology,xu2020-evoexp
0,1.997661,21.832129,19.809110,6.192940,22.616938,51.894794,20.661003,4.028904,1.747370,0.070569,...,2.118876,30.451822,21.495321,0.684262,0.775744,-1.085056,1.275533,0.334654,3.216064,6.973956
1,1.771521,22.697828,19.206188,6.009402,25.816668,54.200546,18.980013,6.507606,1.687792,0.004232,...,11.022372,44.576687,20.686605,0.804477,0.383542,-1.487386,1.393101,0.372090,3.972996,14.250156
2,1.356986,23.600096,21.803297,6.205362,22.387648,55.610287,25.363401,4.146710,2.175165,0.084054,...,3.992358,49.634464,29.742537,0.740055,0.858972,-0.800373,1.305262,0.546611,2.534679,10.201760
3,1.292079,21.815004,21.148670,6.021731,20.890960,48.801975,21.122164,4.587263,2.013699,0.093363,...,4.751814,46.908760,25.168710,0.750065,0.769020,-1.182957,1.296784,0.541955,2.933048,6.631388
4,2.081037,20.933662,20.399752,6.207909,21.624157,52.812386,20.255556,4.082632,1.717985,0.097842,...,1.930887,38.571500,20.837347,0.648204,0.636619,-0.999714,1.272513,0.320140,3.198573,3.725246
